In [2]:
import torch
import torch.nn as nn


In [ ]:
class GlacierSegmentationNetwork(nn.Module):
    """端到端冰川分割网络"""
    def __init__(self, img_channels, dem_channels, num_classes):
        super().__init__()
        # 遥感影像编码器
        self.img_encoder = ResNetBackbone(img_channels)
        
        # DEM处理流
        self.dem_processor = nn.Sequential(
            nn.Conv2d(dem_channels, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            TerrainFeatureExtractor(64),
            ResBlock(128, 256)
        )
        
        # 多级融合模块
        self.fusion_blocks = nn.ModuleList([
            MultiScaleElevationAttention(64, 64),         # 高分辨率特征融合
            CrossModalAttention(128, 128),                # 中分辨率特征融合
            ElevationZoneAttention(256, 256, num_zones=5) # 低分辨率特征融合
        ])
        
        # 解码器
        self.decoder = DeepLabv3PlusDecoder(256, num_classes)
        
        # 后处理优化
        self.post_process = TerrainConstrainedCRF()
    
    def forward(self, img, dem):
        # 提取影像特征
        img_feats = self.img_encoder(img)
        
        # 处理DEM
        dem_feat = self.dem_processor(dem)
        
        # 多级融合
        fused_feats = []
        for i, block in enumerate(self.fusion_blocks):
            if i == 0:
                fused = block(img_feats[i], dem)
            else:
                fused = block(img_feats[i], dem_feat)
            fused_feats.append(fused)
        
        # 解码生成分割图
        seg_logits = self.decoder(fused_feats)
        
        # 后处理优化 (可选)
        seg_map = self.post_process(seg_logits, dem)
        
        return seg_map
